In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.models import Sequential
from sklearn.metrics import accuracy_score, f1_score
from sklearn.pipeline import Pipeline
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.nn import functional as F
from torch import optim
import math



In [ ]:
from tensorflow.keras.datasets import mnist

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
print(x_train[1])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  51 159 253
  159  50   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0  48 238 252 252
  252 237   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0  54 227 253 252 239
  233 252  57   6   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0  10  60 224 252 253 252 202
   84 252 253 122   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0 163 252 252 252 25

In [ ]:
y_train[0]

5

In [ ]:
class Resize():
    def __init__(self):
      self.divisor = None
      self.y = None

    def fit(self, X, y=None):
        self.divisor = 1./255
        self.y = y
        print(y)
        return self

    def transform(self, X):
        if self.y is None:
            print("text")
            # return X * self.divisor
            return (X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)) * self.divisor

        if len(X.shape) < 4:
            y = keras.utils.to_categorical(self.y, len(np.unique(self.y)))
            self.y = None
            return (X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)) * self.divisor, y

        # return (X.reshape(X.shape[0], X.shape[1], X.shape[2], X.shape[3])) * self.divisor, y




In [ ]:
class Conv_model():
    def __init__(self):
        self.model = None

    def fit(self, X, y=None):
        x, y = X
        print(type(x))
        tf_model = keras.Sequential(

        [

        keras.layers.Conv2D(64, (3,3), activation = 'relu', input_shape = (x.shape[1], x.shape[2], x.shape[3])),

        keras.layers.MaxPooling2D((2,2)),

        # keras.layers.Conv2D(64, (3,3), activation = 'relu'),

        # keras.layers.MaxPooling2D((2,2)),

        keras.layers.Flatten(),

        keras.layers.Dense(64, activation = 'relu'),

        keras.layers.Dropout(0.1),

        keras.layers.Dense(10, activation = 'softmax')
        ])

        tf_model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

        tf_model.fit(x, y, epochs = 1)

        self.model = tf_model

        return self


    def transform(self, X, y):
        return X


    def predict(self, X):
        return np.array(np.argmax(self.model.predict(X), axis = 1))


    def score(self, X, y):

          predict = self.predict(X)

          return accuracy_score(y, predict), f1_score(y, predict, average = 'weighted')




In [ ]:
_pipeline = Pipeline([('resize', Resize()), ('model', Conv_model())])

In [ ]:
_pipeline.fit(X= x_train, y = y_train)

[5 0 4 ... 5 6 8]
<class 'numpy.ndarray'>


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 60s 31ms/step - accuracy: 0.8980 - loss: 0.3387


Pipeline(steps=[('resize', <__main__.Resize object at 0x786b29d9e980>),
                ('model', <__main__.Conv_model object at 0x786b29d9ebf0>)])

In [ ]:
_pipeline.score(x_test, y_test)

text
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step


(0.9804, 0.9803945552980134)

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, num_classes, hidden_size):
        super(NeuralNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 14 * 14, hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = self.pool1(out)
        out = torch.flatten(out, start_dim=1)
        out = torch.relu(self.fc1(out))
        out = torch.softmax(self.fc2(out), dim=1)
        return out

In [ ]:
class TorchNet():
    def __init__(self):
        self.model = None
        self.batch_size = 32

    def fit(self, X, y=None):

        model = NeuralNet(hidden_size=64, num_classes=10)
        loss = nn.CrossEntropyLoss()
        optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.8)
        epochs = 2


        x, y = X
        x = torch.from_numpy(x).float()
        y = torch.from_numpy(y).float()
        x = x.permute(0, 3, 1, 2)
        print(x.shape)
        print(len(x))


        model.train()
        sumL, sumA, numB = 0, 0, math.ceil(len(x) / self.batch_size)
        # list_predict = []
        for epoch in range(epochs):
            for i in range(0, numB * self.batch_size, self.batch_size):
                xb = x[i:i + self.batch_size]
                yb = y[i:i + self.batch_size]

                y_pred = model(xb)
                L = loss(y_pred, yb)
                optimizer.zero_grad()
                L.backward()
                optimizer.step()

                # sumL += L.item()
                # _, predicted = torch.max(y, 1)
            # sumA += (predicted == yb).float().mean()

        self.model = model
        return self


    def transform(self, x):
        return x

    def predict(self, X):

        x = torch.from_numpy(X).float()
        x = x.permute(0, 3, 1, 2)
        numB = math.ceil(len(x) / self.batch_size)
        list_predict = []
        self.model.eval()
        for i in range(0, numB * self.batch_size, self.batch_size):
            xb = x[i:i + self.batch_size]
            y = self.model(xb)
            _, predicted = torch.max(y, 1)
            list_predict.extend(predicted.detach().numpy())

        return np.array(list_predict)


    def score(self, X, y):

          predict = self.predict(X)

          return accuracy_score(y, predict), f1_score(y, predict, average = 'weighted')


In [ ]:
pipeline = Pipeline([('resize', Resize()), ('model', TorchNet())])

In [ ]:
pipeline.fit(X= x_train, y = y_train)

[5 0 4 ... 5 6 8]
torch.Size([60000, 1, 28, 28])
60000


Pipeline(steps=[('resize', <__main__.Resize object at 0x786b2fa229b0>),
                ('model', <__main__.TorchNet object at 0x786b2fa20670>)])

In [ ]:
pipeline.score(x_test, y_test)

text


(0.9624, 0.9626191197438476)